In [40]:
from flask import Flask, request, jsonify
import base64
import tempfile
import os
from bob.io.audio import reader
from utils import configuration, silence_detection
import numpy as np
from datetime import datetime
from scipy.spatial import distance

In [6]:
config = configuration()

In [ ]:
'''
sample json
 {
     task : 'enroll/verify',
     aud_b64 : 'base64 audio of speech',
     spkr_id : '16bit hash of speaker id'
 }
'''

In [ ]:
class iVector_model:
    def __init__(self,data):
        self.data = data
        self.fubm = open(config['model_files']['full_GMM-UBM']).read()
        self.ivector_extractor = open(config['model_files']['iVector']).read()

    def get_mfcc(self):
        norm_speech = self.data.load()[0]/2**15
        speech_segs = silence_detection(self.data)
        
        mfcc = []
        for seg in speech_segs:
            seg_mfcc = bob.kaldi.mfcc(norm_speech[seg[0]:seg[1]])
            mfcc.append(seg_mfcc)

        mfcc = np.vstack(mfcc)
        returrn mfcc
        
    def get_iVector(self):
        mfcc = self.get_mfcc()
        if mfcc_feat.shape[0] > 2000+1: # utterances longer than 30s are trimmed to 30s
            mfcc_feat = mfcc_feat[0,2000+1,:]
        
        iVector = bob.kaldi.ivector_extract(self.mfcc_feat,
                                            self.fubm,
                                            self.ivector_extractor,
                                            num_gselect=config['iVector_extract_params']['num_gselect'],
                                            min_post=config['iVector_extract_params']['min_post'],
                                            posterior_scale=config['iVector_extract_params']['posterior_scale'])
        return iVector

    def enroll(self,spkr_id):
        if not(spkr_id in os.listdir(config['enrolled_speakers']['path'])):
            iVector = self.get_iVector()
            
            os.makedirs(config['enrolled_speakers']['path'])
            spkr_iVec_path = os.path.join(config['enrolled_speakers']['path'],f'{spkr_id}_{str(datetime.now())}_iVec.npy')
            np.save(os.path.join(spkr_iVec_path,iVector)
            return True
            
        else:
            return False

    def verify(self,spkr_id):
        if spkr in os.listdir(config['enrolled_speakers']['path']):
            spkr_iVec_path = os.path.join(config['enrolled_speakers']['path'],
                                          os.listdir(config['enrolled_speakers']['path'])[0]) # default first iVec file in dir
            enroll_iVector = np.load(spkr_iVec_path)
            verify_iVector = self.get_iVector()
                    
            return 1-distance.cosine(enroll_iVector,verify_iVector)
        else:
            return False

In [6]:
def decode_base64(base64string):
    b64_byte = base64.b64decode(base64string)
    wav_temp = tempfile.TemporaryFile()
    
    with open("./" + str(wav_temp.name) ,'wb') as wav: 
        wav.write(b64_byte)
    
    data = bob.io.audio.reader(str(wav_temp.name))
    
    if data.rate!=16000:
        return {'status':'fail',
                'msg':'acceptable sampling rate is 16kHz',
                'wav_temp' :wav_temp, 
                'data' : data}

    elif data.load().shape[0] != 1:
        return {'status' : 'fail',
                'msg' : 'mono channel audio required',
                'wav_temp' : wav_temp,
                'data' : data }
    

    return {'status': 'pass',
            'msg' : 'acceptable audio',
            'wav_temp' : wav_temp,
            'data' : dat ccca }

In [ ]:
def verifySpeaker(payload):
    [task,aud_b64,spkr_id] = paylod.values()
    [status,msg,_,data].values() = decode_base64(base64string)
    
    if status = 'fail':
        return {'status': status,'msg' : msg}
    
    if task == 'enroll':
        model = iVector_model(data)
        model.enroll(spkr_id)
    
    elif task == 'verify':
    
    else:
        return {}
   
     task : 'enroll/verify',
     aud_b64 : 'base64 audio of speech',
     spkr_id : '16bit hash of speaker id'

    
    
    
    

In [15]:
import numpy as np
from datetime import datetime

In [16]:
arr = np.ones((12,12))
path = './enrolled_speakers/spkr_r12079'
# os.makedirs(path)
np.save(os.path.join(path,f'r12079_{str(datetime.now())}_iVec.npy'),arr)

In [18]:
'spkr_r12079' in os.listdir('./enrolled_speakers/') 

True

In [ ]:
payload = {''}